In [1]:
import json
import re

# 파일 경로
mental_disorders_path = '질병_증상_치료법_치료제.json'
medications_path = '치료제_부작용.json'
synonyms_path = 'final_synonym_dict.json'

# JSON 로드
with open(mental_disorders_path, 'r', encoding='utf-8') as f:
    mental_disorders = json.load(f)
with open(medications_path, 'r', encoding='utf-8') as f:
    medications = json.load(f)
with open(synonyms_path, 'r', encoding='utf-8') as f:
    synonyms = json.load(f)

# 약물 동의어 맵 생성
def build_medication_synonym_map(medications):
    med_synonyms = {}
    for med in medications:
        med_field = med.get('약물 (일반명/상품명)', '')
        match = re.match(r'(.+?)\s*\((.+?)\)', med_field)
        if match:
            names = [match.group(1).strip(), match.group(2).strip()]
        else:
            names = [med_field]
        med_synonyms[names[0]] = names
    return med_synonyms

med_synonym_map = build_medication_synonym_map(medications)
for key, values in med_synonym_map.items():
    synonyms[key] = list(set(synonyms.get(key, []) + values))

# 동의어 매칭
def match_synonym(user_input, target_key):
    return any(re.search(rf'\b{re.escape(word)}\b', user_input) for word in synonyms.get(target_key, [target_key]))

# 민감/성격 트리거
sensitive_triggers = ['죽고 싶', '자살', '극단', '해치']
personality_triggers = ['충동', '대인관계', '분노', '불안정']
med_keywords = ['약', '약물', '치료', '복용', '추천', '먹는 약']

# 챗봇 함수
def chatbot(user_input):
    try:
        user_input_clean = re.sub(r'[^\w\s]', '', user_input.strip())

        # 민감 질문 처리
        if any(trigger in user_input_clean for trigger in sensitive_triggers):
            return "⚠️ 위험한 생각이 드신다면 가까운 사람이나 전문가에게 꼭 도움을 요청하세요. 💛 당신은 혼자가 아닙니다."

        # 부작용 질문
        if '부작용' in user_input_clean:
            for med in medications:
                med_field = med.get('약물 (일반명/상품명)', '')
                med_names = med_synonym_map.get(med_field.split()[0], [med_field])
                if any(med_name in user_input_clean for med_name in med_names):
                    return f"{med_field} 부작용:\n- {med.get('부작용 상세', '부작용 정보 없음')}"
            return "앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요."

        # 약물 질문
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            if match_synonym(user_input_clean, disorder_name):
                treatments = disorder.get('치료제', '')
                symptoms = disorder.get('증상', '')
                if any(kw in user_input_clean for kw in med_keywords) and treatments:
                    return f"{disorder_name}에 사용되는 약물:\n- {treatments}\n📌 특징 증상:\n- {symptoms}"

        # 증상 질문
        matched_diseases = []
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            symptoms = disorder.get('증상', '')
            symptom_words = re.split(r'[·, ]', symptoms)
            match_count = sum(1 for word in symptom_words if word and word in user_input_clean)
            if match_synonym(user_input_clean, disorder_name) or match_count >= 1:
                matched_diseases.append((disorder_name, symptoms))

        # 성격장애 별도 처리
        trigger_count = sum(1 for trigger in personality_triggers if trigger in user_input_clean)
        if trigger_count >= 2:
            disorder = next((d for d in mental_disorders if d.get('질병') == '성격장애'), {})
            matched_diseases.append(('성격장애', disorder.get('증상', '증상 정보 없음')))

        if matched_diseases:
            response = "🔍 관련 질병 및 특징:\n"
            for name, symptoms in matched_diseases[:3]:
                response += f"- {name}: {symptoms}\n"
            response += "더 궁금한 게 있으면 편하게 물어봐 주세요 😊"
            return response

        # fallback: 감정 케어
        for key, words in synonyms.items():
            if any(word in user_input_clean for word in words):
                disorder = next((d for d in mental_disorders if d.get('질병') == key), {})
                symptoms = disorder.get('증상', '증상 정보 없음')
                return f"{key} 관련된 감정이 느껴져요.\n📌 특징:\n- {symptoms}\n더 궁금한 게 있으면 물어봐 주세요 😊"

        return "앗, 아직 해당 질문에 대한 답변을 준비하지 못했어요. 조금 더 구체적으로 말씀해 주세요 😊"

    except Exception as e:
        return f"앗, 오류가 발생했어요! 잠시 후 다시 시도해 주세요.\n(오류 내용: {str(e)})"


In [2]:
test_questions = [
    "플루옥세틴 부작용 뭐야?",
    "에스시탈로프람 먹으면 무슨 부작용 있어?",
    "자낙스 부작용 알려줘",
    "공황장애 약물 뭐 있어?",
    "불안할 때 먹는 약 추천해줘",
    "우울증에 쓰는 약 알려줘",
    "불면증에 좋은 약은 뭐야?",
    "ADHD 약물 추천해줄 수 있어?",
    "알츠하이머 약물 치료법 있어?",
    "양극성장애 약물 뭐가 있어?",
    "불안장애 증상 알려줘",
    "우울증 증상 뭐야?",
    "공황장애 증상 궁금해",
    "경계성 성격장애 특징은?",
    "자꾸 폭식하게 돼. 왜 그래?",
    "최근에 잠이 잘 안 와",
    "기분이 너무 오락가락해",
    "집중이 잘 안 돼",
    "요즘 무기력해",
    "나 요즘 예민해졌어",
    "요즘 너무 불안해",
    "너무 우울해",
    "최근에 너무 충동적이야",
    "나는 정신과 가야 할까?",
    "나 요즘 너무 예민해서 힘들어",
    "아무것도 하기 싫어",
    "요즘 사람 만나는 게 무서워",
    "감정 조절이 안 돼",
    "아무 생각이 없고 멍해",
    "죽고 싶다는 생각이 자꾸 들어"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {chatbot(q)}\n")


Q: 플루옥세틴 부작용 뭐야?
A: 플루옥세틴 (프로작) 부작용:
- 불안, 불면, 메스꺼움, 두통, 성기능 장애, 식욕 저하, 입 마름, 발한, 설사, 떨림

Q: 에스시탈로프람 먹으면 무슨 부작용 있어?
A: 에스시탈로프람 (렉사프로) 부작용:
- 메스꺼움, 불안, 어지럼증, 졸음, 불면, 성기능 장애, 입 마름, 발한, 두통, 설사

Q: 자낙스 부작용 알려줘
A: 앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요.

Q: 공황장애 약물 뭐 있어?
A: 공황장애에 사용되는 약물:
- SSRI(설트랄린, 파록세틴), 벤조디아제핀(알프라졸람)
📌 특징 증상:
- 갑작스러운 극심한 공포, 심계항진, 발한, 떨림, 숨가쁨, 흉통, 메스꺼움, 현기증, 비현실감, 죽음에 대한 공포, 발작 후 극도의 피로감

Q: 불안할 때 먹는 약 추천해줘
A: 성격장애에 사용되는 약물:
- SSRI(충동성·우울), 항정신병 약물(충동·공격성)
📌 특징 증상:
- 경직된 사고·행동 패턴, 대인관계 문제, 사회적 부적응, 충동성, 정서 불안정, 타인·자신에 대한 왜곡된 인식, 행동이 반복적이고 고착적임

Q: 우울증에 쓰는 약 알려줘
A: 성격장애에 사용되는 약물:
- SSRI(충동성·우울), 항정신병 약물(충동·공격성)
📌 특징 증상:
- 경직된 사고·행동 패턴, 대인관계 문제, 사회적 부적응, 충동성, 정서 불안정, 타인·자신에 대한 왜곡된 인식, 행동이 반복적이고 고착적임

Q: 불면증에 좋은 약은 뭐야?
A: 성격장애에 사용되는 약물:
- SSRI(충동성·우울), 항정신병 약물(충동·공격성)
📌 특징 증상:
- 경직된 사고·행동 패턴, 대인관계 문제, 사회적 부적응, 충동성, 정서 불안정, 타인·자신에 대한 왜곡된 인식, 행동이 반복적이고 고착적임

Q: ADHD 약물 추천해줄 수 있어?
A: 성격장애에 사용되는 약물:
- SSRI(충동성·우울), 항정신병 약물(충동·공격성)
📌 특징 증상:
- 경직된 사고·행동 패턴, 대인관계 문제, 사회적 부적응, 충동성, 정서 불안정,